In [8]:
import requests
from bs4 import BeautifulSoup
from time import sleep 
import csv
import pandas as pd

In [2]:
url = 'https://arxiv.org/list/cs/pastweek?skip=0&show=100'
page = requests.get(url=url)
soup = BeautifulSoup(page.text, 'html.parser')

In [3]:
assert url
assert page
assert soup

In [4]:
content = soup.dl
list_id = []
for link in content.find_all('a', title='Abstract'):
    list_id.append(link['href'])

In [5]:
list_id

['/abs/2403.00762',
 '/abs/2403.00758',
 '/abs/2403.00752',
 '/abs/2403.00745',
 '/abs/2403.00743',
 '/abs/2403.00742',
 '/abs/2403.00737',
 '/abs/2403.00729',
 '/abs/2403.00725',
 '/abs/2403.00724',
 '/abs/2403.00720',
 '/abs/2403.00717',
 '/abs/2403.00715',
 '/abs/2403.00712',
 '/abs/2403.00704',
 '/abs/2403.00696',
 '/abs/2403.00691',
 '/abs/2403.00690',
 '/abs/2403.00689',
 '/abs/2403.00686',
 '/abs/2403.00685',
 '/abs/2403.00682',
 '/abs/2403.00680',
 '/abs/2403.00675',
 '/abs/2403.00674',
 '/abs/2403.00673',
 '/abs/2403.00669',
 '/abs/2403.00668',
 '/abs/2403.00665',
 '/abs/2403.00663',
 '/abs/2403.00662',
 '/abs/2403.00646',
 '/abs/2403.00645',
 '/abs/2403.00644',
 '/abs/2403.00643',
 '/abs/2403.00642',
 '/abs/2403.00641',
 '/abs/2403.00633',
 '/abs/2403.00632',
 '/abs/2403.00631',
 '/abs/2403.00628',
 '/abs/2403.00625',
 '/abs/2403.00623',
 '/abs/2403.00622',
 '/abs/2403.00621',
 '/abs/2403.00613',
 '/abs/2403.00611',
 '/abs/2403.00607',
 '/abs/2403.00606',
 '/abs/2403.00598',


In [6]:
list_title = []
list_author = []
list_subjects = []
list_abstract = []

for pid in list_id:
    for _ in range(4):
        sleep(5)
        try:
            url = 'https://arxiv.org' + pid
            page = requests.get(url=url, timeout=10)  # 设置超时时间为10秒
            soup = BeautifulSoup(page.text, 'html.parser')

            assert url
            assert page
            assert soup

            title = soup.find('h1', class_='title mathjax').text.strip().replace('Title:', '')

            subjects = soup.find('span', class_='primary-subject').text.strip()

            authors = soup.find('div', class_='authors').text.strip().replace('Authors:', '')

            abstract = soup.find('blockquote', class_='abstract mathjax').text.strip().replace('Abstract:', '')

            list_title.append(title)
            list_subjects.append(subjects)
            list_author.append(authors)
            list_abstract.append(abstract)

            print('paper ---' + pid + '--- has been done')
            
            break
        except requests.Timeout:
            print(f"Timeout error occurred while fetching data from {url}")
        except Exception as e:
            print(f"An error occurred: {e}")

paper ---/abs/2403.00762--- has been done
paper ---/abs/2403.00758--- has been done
paper ---/abs/2403.00752--- has been done
paper ---/abs/2403.00745--- has been done
paper ---/abs/2403.00743--- has been done
paper ---/abs/2403.00742--- has been done
Timeout error occurred while fetching data from https://arxiv.org/abs/2403.00737
Timeout error occurred while fetching data from https://arxiv.org/abs/2403.00737
paper ---/abs/2403.00737--- has been done
Timeout error occurred while fetching data from https://arxiv.org/abs/2403.00729
Timeout error occurred while fetching data from https://arxiv.org/abs/2403.00729
paper ---/abs/2403.00729--- has been done
paper ---/abs/2403.00725--- has been done
paper ---/abs/2403.00724--- has been done
paper ---/abs/2403.00720--- has been done
paper ---/abs/2403.00717--- has been done
Timeout error occurred while fetching data from https://arxiv.org/abs/2403.00715
Timeout error occurred while fetching data from https://arxiv.org/abs/2403.00715
paper ---/

In [13]:
df = pd.DataFrame({
    "Title": list_title,
    "Author": list_author,
    "Subjects": list_subjects,
    "Abstract": list_abstract
})

df.to_csv('output.csv', index=False)